In [4]:
import os

In [34]:
# Set the OpenAI API key in the Python environment.
# `os` is already imported in a later cell, so we can use it here without re-importing.
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

# Optional confirmation (does not print the secret)
print("OPENAI_API_KEY set in os.environ")

OPENAI_API_KEY set in os.environ


In [15]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("../data_new/speaker_texts/train", glob="**/*.txt")

In [13]:
loader

In [17]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.9 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 13.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.2 MB/s  0:00:00
  DEPRECATION: Building 'langdetect' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'langdetect'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993284 sha256=7a025375cf3433c30256008b7ab9ce55c83c7a2cd33c1c2edcdc489006624475
  Stored in directory: /home/e

In [18]:
docs = loader.load()

In [19]:
len(docs)

33

In [20]:
from pathlib import Path

for d in docs:
    # e.g. source = ".../Odysseus.txt" → "Odysseus"
    fname = Path(d.metadata["source"]).stem
    d.metadata["character"] = fname

In [21]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()  # uses OPENAI_API_KEY

vectorstore = FAISS.from_documents(docs, embeddings)


In [29]:
from typing import Tuple, List
from langchain_core.documents import Document

def predict_speaker(
    line: str,
    k: int = 33
) -> Tuple[str, List[Document]]:
    """
    Returns:
      - predicted_character: character name from best-matching document
      - results: top-k matching documents for inspection
    """
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    results: List[Document] = retriever.get_relevant_documents(line)

    if not results:
        return "UNKNOWN", []

    top = results[0]
    predicted_character = top.metadata.get("character", "UNKNOWN")
    return predicted_character, results


In [32]:
query_line = "Ruthlessness is mercy upon ourselves 'Cause you fight to save lives"

predicted_character, matches = predict_speaker(query_line)

print("Predicted speaker:", predicted_character)
print("--- top hit meta ---")
print(matches[0].metadata)
print("--- snippet of that doc ---")
print(matches[0].page_content[:500])  # just to sanity check


Predicted speaker: Laestrygonians
--- top hit meta ---
{'source': '../data_new/speaker_texts/train/Laestrygonians.txt', 'character': 'Laestrygonians'}
--- snippet of that doc ---
You are far too nice

Unless, of course, you apologize

Ruthlessness is (Captain) mercy upon our- Captain

Poseidon, Poseidon, Poseidon, Poseidon, Poseidon, Poseidon, Poseidon

Ruthlessness is (Captain) mercy upon our- Captain

Had you just killed my son

Ruthl?ssness is mercy upon ourselv?s

I've gotta make you bleed

You reveal your name

Unlike you I've got no mercy left to give 'cause

You are the worst kind of good

That's what I hate

Ruthlessness is mercy-

But before you go, I need to ma


In [33]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4.1-mini")

prompt = ChatPromptTemplate.from_template("""
You are given some example lines spoken by different characters from a script.
Each block is labeled with the character name.

Use the writing STYLE and content to decide who most likely says the QUERY line.
Respond with ONLY the character name.

CONTEXT:
{context}

QUERY LINE:
"{query}"
""")

def predict_speaker_llm(line: str, k: int = 3) -> str:
    results = vectorstore.similarity_search(line, k=k)

    context_blocks = []
    for doc in results:
        char = doc.metadata.get("character", "UNKNOWN")
        # We can just show the first few hundred chars of each file as style examples
        context_blocks.append(f"[{char}]\n{doc.page_content[:800]}\n")
    context = "\n".join(context_blocks)

    chain = prompt | llm
    resp = chain.invoke({"context": context, "query": line})
    return resp.content.strip()

speaker = predict_speaker_llm(query_line)
print("LLM-predicted speaker:", speaker)


LLM-predicted speaker: Poseidon
